Openvino 2021.4 is used here since I am using an OAK-D SDK on the robot that is compatible with 2021.4.


In [4]:
!pip install openvino-dev[tensorflow2,onnx]==2021.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.1 MB/s 
     |████████████████████████████████| 1.9 MB 26.6 MB/s 
     |████████████████████████████████| 3.1 MB 37.8 MB/s 
     |████████████████████████████████| 3.3 MB 32.7 MB/s 
     |████████████████████████████████| 28.2 MB 1.3 MB/s 
     |████████████████████████████████| 99 kB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 14.8 MB 41.8 MB/s 
     |████████████████████████████████| 1.3 MB 41.8 MB/s 
     |████████████████████████████████| 15.6 MB 132 kB/s 
     |████████████████████████████████| 9.5 MB 48.5 MB/s 
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 517 kB 57.2 MB/s 
     |█████████████████████████████

In [5]:
import tensorflow as tf


In [ ]:
!mo -h

In [6]:
model = tf.keras.models.load_model('/content/mypilot.h5')
tf.keras.models.save_model(model, 'model2')

Use openvino model optimzer to convert the model
* Scale the input from 0-255 to 0-1.0
* Apply RGB -> GBR
* Specify the shape to avoid MyriadX blob conversion failure (later step)

In [7]:
!mo --saved_model_dir /content/model2  --input_shape [1,120,160,3] --scale 255 --reverse_input_channels

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	None
	- Path for generated IR: 	/content/.
	- IR output name: 	saved_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,120,160,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	255.0
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	True
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use configuration file used to generate the model with Object Detection API: 	None
	- Use

The above output was used with http://blobconverter.luxonis.com/ to convert to a Myriad X blob for OpenVino 2021.4 (which is the version compatible with the OAK-1 SDK I am currently using.



From here onward I tried converting to Onnx format and then using that with the Myriad X converter.  I tried this path since the OpenVino format initially failed to convert to a Myriad X blob because the input shape was not specified above.  I also had a failure since the above was using the 2022.1 Openvino, but specifying 2021.4 when converting to Myriad X.  I changed the above to use 2021.4.



In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
tensorflow 2.5.3 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
openvino 2022.1.0 requires numpy<1.20,>=1.16.6, but you have numpy 1.21.6 which is incompatible.
openvino-dev 2022.1.0 requires numpy<1.20,>=1.16.6, but you have numpy 1.21.6 which is incompatible.
openvino-dev 2022.1.0 requires numpy<=1.21,>=1.16.6; python_version > "3.6", but you have

In [ ]:
!pip install -U tf2onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m tf2onnx.convert --saved-model model2 --output model2.onnx

2022-09-07 03:12:17.554797: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-09-07 03:12:19.278287: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-09-07 03:12:19.287076: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-07 03:12:19.287133: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a07c05d8a7c8): /proc/driver/nvidia/version does not exist
2022-09-07 03:12:19.287561: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is op